In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from auction.operations import *
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')
print('STORE_APP_ID', os.environ.get('STORE_APP_ID'))

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

STORE_APP_ID 2779


In [3]:
staking_app = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
team_wallet = Account.from_mnemonic(os.environ.get("TEAM_MN"))

creator = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
seller = Account.from_mnemonic(os.environ.get("SELLER_MN"))

bidder = Account.from_mnemonic(os.environ.get("BUYER_MN"))
bidder1 = Account.from_mnemonic(os.environ.get("BUYER1_MN"))
bidder2 = Account.from_mnemonic(os.environ.get("BUYER2_MN"))

print(f"Staking App Address: {staking_app.get_address()}")
print(f"Team wallet Address: {team_wallet.get_address()}")
print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")
print(f"Bidder Address: {bidder.get_address()}")
print(f"Bidder1 Address: {bidder1.get_address()}")
print(f"Bidder2 Address: {bidder2.get_address()}")   
  


Staking App Address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Team wallet Address: UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
Bidder Address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Bidder1 Address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Bidder2 Address: 742N2DLQ6NUUQXNUTNIJHYXYGL6BWG7C2XSQDHLFJN2NFYDVAYYUAQ36NQ


In [4]:
#seller_info = get_account_info(client, seller.get_address())
#print('seller_info', seller_info)

In [5]:
deleteApps(client, [3177, 3179, 3200, 3202, 3223, 3225, 3246, 3248], creator)
# optoutApps(client, [], seller)
# optoutApps(client, [], bidder)
# optoutApps(client, [], bidder1)
# optoutApps(client, [], bidder2)

Waiting for confirmation...
Transaction I4HN3ORZE6K2AFY4N57BY4QIKJTEGN5ZYFDC2XJMGZWQNOZN7CMA confirmed in round 126209.
Waiting for confirmation...
Transaction LCT2XLXFLSKCQU5V7BK5J4IS73SRVJ3VVJH2YRZAJYOQUCZNOVCQ confirmed in round 126211.
Waiting for confirmation...
Transaction WLTBUAF4YGBO257WMDQ3IGBRPEWKBBSKSBTR7KCGB64RVNV46ICQ confirmed in round 126213.
Waiting for confirmation...
Transaction L3SWFZWG2B5UC2YR6R6ZQA3YVKM6N6KUHFB2OUSMNYTXHJSNQBEQ confirmed in round 126215.
Waiting for confirmation...
Transaction KD4K2SRHW4KSTCVZO64GIKQQ4XUPQFRPUL7733WGSXFRMJUFDZJA confirmed in round 126217.
Waiting for confirmation...
Transaction O5PAGA2VIXTRGIBRFP64VMCIP5ZUEVXLLY2UAWSMNZCCQJS5FY2A confirmed in round 126219.
Waiting for confirmation...
Transaction KU4FEXROB4GCBIXUSTWS6QMUBFAH2PODLQB5YOBGQWKDN3JSZHDA confirmed in round 126221.
Waiting for confirmation...
Transaction UV7HSXNFAPM74JCEGDCNFK3KZVL7GFS2TNDXCNCGOICI6KKBAMQA confirmed in round 126223.


Alice is generating an example token...

In [6]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(creator, 1000000000000000000, 3, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")
#send_asset(client, asset_id, 500000000000000, creator, seller)

The token id is: 2989


In [7]:
start_time = int(time()) + 10
end_time = start_time + 30
print(f"start_time", start_time)
reserve = 1_000_000  # 1 Algo
token_amount = 100
increment = 100_000  # 0.1 Algo

start_time 1641486284


In [8]:
storeApp = StoringPool(client=client, creator=creator)

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID

# optout_app(client, storeAppID, seller)
# optout_app(client, storeAppID, bidder)
# optout_app(client, storeAppID, bidder1)
# optout_app(client, storeAppID, bidder2)

storeApp.create_app()
storeAppID = storeApp.app_id

print(f"storeAppID", storeAppID)
print(b"storeApp address", get_application_address(storeAppID))


Waiting for confirmation...
Transaction MYPFDEYERNITHGKBFLF6BX7ZMHGLOSMSFVYZHYFU5VPII5VUO2WQ confirmed in round 126225.
Store App ID: 3283
Store App address: FYB4HRQYVCKRFESMCJDQIYD6V4RCTXNVZ3CLQV7MLTCTFTTNGGBX5UKVCM
Waiting for confirmation...
Transaction NRDSPQDP2EHRGSQJ6TSIGMJ36EY4BHD26YAOHOEKQKJEQKYMTSNA confirmed in round 126227.
storeAppID 3283
b'storeApp address' FYB4HRQYVCKRFESMCJDQIYD6V4RCTXNVZ3CLQV7MLTCTFTTNGGBX5UKVCM


"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [9]:

print(f"seller address: {seller.get_address()}")
appID = create_auction_app(
    client=client,
    creator=creator,
    staking_address=staking_app.get_address(),
    team_wallet_address=team_wallet.get_address(),
    store_app_id=storeAppID
)

#appID = 2843
print(f"Auction App ID: {appID}")
print(f"Auction App Address: {get_application_address(appID)}")
print("Latest time:", get_app_global_state(client, appID))

seller address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
Waiting for confirmation...
Transaction CHAOOQ5UCO6DMHSUCKE5BHGUSYWF5BS56IPZDSM7F5E6PRY4DG6Q confirmed in round 126229.
Waiting for confirmation...
Transaction XRSKUVOZVILXZIFLUERLJ7OP42VVTKVUGZ73LDSYLOPYMYCAFELA confirmed in round 126231.
Auction App ID: 3285
Auction App Address: 2RULKVZUQVGHARJYPMJUVJ7NS2P6F5QF6U2XZY54TOHZ4NYHPK6EIYS5SQ
Latest time: {b'SA_ADDR': b'\xe4\xe7\xfe\xeb\xfc h\x02\xf3\xc2D\xe8\xbf\x1e\x10\xe0\xb8=\x99\xdf1\x1dX\xe6P\xce\xfb[\xc8\xd1\xa9\xd3', b'SA_ID': 3283, b'TW_ADDR': b"\xa0\x81}\xb3\xa7\x97\xdfk,>8\xc4\xcd;\xe9'\x0e-\xc4\xc1{lX\xbf\x92\xc5=\xac\xcb\xad\x94\xb4"}


Alice is setting up and funding token auction...

In [10]:
storeApp.set_up(trade_app_id=100, bid_app_id=101, auction_app_id=appID)

Waiting for confirmation...
Transaction ZVBY54U4O6UG3URFVTVHKHGAZK54W5XQ5FPZ2LUI5TRD66TVWF6A confirmed in round 126233.


In [11]:
auction_index = setup_auction_app(
    client=client,
    app_id=appID,
    seller=seller,
    token_id=asset_id,
    token_amount=token_amount,
    start_time=start_time,
    end_time=end_time,
    reserve=reserve,
    min_bid_increment=increment
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

store_app_id 3283
seller CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA opt in app 3283
Waiting for confirmation...
Transaction FYOKP2P6JOK6SVSCJGDXNRSEBGEQT5VEAZJD445LCEPT3C427B5Q confirmed in round 126235.
new address: DIUGRE7GC4CCUCCE3SCQFM4Q5ELCNEU4EPFKAW72G6MAHU262N2W4ICAXU
new private_key: yz9d7bSjn9CE79wPHNdWNP1poz68ctNq8MzPhvwZp3saKGiT5hcEKghE3IUCs5DpFiaSnCPKoFv6N5gD017TdQ==
new passphrase: west insane issue deputy dismiss crouch label table limit fragile clock spy stadium potato tiger increase foil script toy hollow weekend shoe tag about news
Waiting for confirmation...
Transaction HFMNLL7DTIONOSFZSD7FF5RHKPLRTYY2LERINIUGPGP2IMLNBJ4Q confirmed in round 126237.
Waiting for confirmation...
Transaction AM7C7WTYJTVPO2SCOBTHLL2C4W5Y5XLM67UFPB63D37DOVOLAKVQ confirmed in round 126239.
Waiting for confirmation...
Transaction PPROJL6VE2CGW4SZFAQJZWO3BS3R5OWZDBVQENLHZ2BUJCTCGSWA confirmed in round 126241.
Waiting for confirmation...
Transaction CNOWSATEQTUI64A6FB3ABVTSHSWEA

In [12]:
# _, last_round_time = get_last_block_timestamp(client)
# print(f"start_time", start_time)
# print(f"last_round_time", last_round_time)
# if last_round_time < start_time + 5:
#     sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 200000, 2989: 100}


In [13]:
bidAmount = reserve + 4000
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

Carla wants to bid on token, her algo balance:  10028777475452  algos
Carla is placing bid for:  1004000  algos


In [14]:
_, lastRoundTime = get_last_block_timestamp(client)
print(f"lastRoundTime", lastRoundTime)
place_bid(client=client, app_id=appID, auction_index=auction_index, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

lastRoundTime 1641486350
Waiting for confirmation...
Transaction FFO6TE2YRLOQPC3EMPWD3Y52H4WEC7UYURLUOA6MUHRGSJEWWZIA confirmed in round 126245.
prev_bid_leader None
Waiting for confirmation...
Transaction NMWLQWL5JEE2FB66CLT4DAHXQWDEPGHAOOZFMLIHJ4EM3LO2TNKQ confirmed in round 126247.
Carla is opting into token with id: 2989


In [15]:
bidAmount = reserve + 300_000
place_bid(client=client, app_id=appID, auction_index=auction_index, bidder=bidder1, bid_amount=bidAmount)

bidAmount = reserve + 500_000
place_bid(client=client, app_id=appID, auction_index=auction_index, bidder=bidder2, bid_amount=bidAmount)


Waiting for confirmation...
Transaction HM5UTY7KTZDE3KDPQNVZ2RTBU4HBMW7FMPEYU3E2N64DJ36V4R6A confirmed in round 126249.
prev_bid_leader X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Waiting for confirmation...
Transaction F3ZNNI3DMLH5AGRIIA47JDNMSMRY3YKWJ7DO7QCU5MYUACZ4APTA confirmed in round 126251.
Waiting for confirmation...
Transaction MRTZV4NJ3BC3GCUKCEHZO5FIUCZQALTO3EWUJ5O764RCZDG3Y3AA confirmed in round 126253.
prev_bid_leader LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Waiting for confirmation...
Transaction XOP7GX3L7D74RHCSW5WR7G5SIUANAVBJCAZ3UJF64S5MPBTRACQA confirmed in round 126255.


True

Alice is closing out the auction....

In [16]:
# _, lastRoundTime = get_last_block_timestamp(client)
# if lastRoundTime < end_time + 5:
#     sleep(end_time + 5 - lastRoundTime)

close_auction(client, appID, auction_index, seller)

app_global_state {b'SA_ADDR': b'\xe4\xe7\xfe\xeb\xfc h\x02\xf3\xc2D\xe8\xbf\x1e\x10\xe0\xb8=\x99\xdf1\x1dX\xe6P\xce\xfb[\xc8\xd1\xa9\xd3', b'SA_ID': 3283, b'TW_ADDR': b"\xa0\x81}\xb3\xa7\x97\xdfk,>8\xc4\xcd;\xe9'\x0e-\xc4\xc1{lX\xbf\x92\xc5=\xac\xcb\xad\x94\xb4"}
['DIUGRE7GC4CCUCCE3SCQFM4Q5ELCNEU4EPFKAW72G6MAHU262N2W4ICAXU', '742N2DLQ6NUUQXNUTNIJHYXYGL6BWG7C2XSQDHLFJN2NFYDVAYYUAQ36NQ', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU', '4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64']
Waiting for confirmation...
Transaction H4OC5NHILT3UNOA3XCQBD4CHU5CEZBKOFZ7APPMFE4WY5EM5LZKQ confirmed in round 126257.


True

In [17]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)
print(f"storeAppID", storeAppID)
#assert actualAppBalances == expectedAppBalances

from utils import get_app_local_state
print(get_app_local_state(client, storeAppID, seller.get_address()))
print(get_app_local_state(client, storeAppID, bidder.get_address()))
print(get_app_local_state(client, storeAppID, bidder1.get_address()))
print(get_app_local_state(client, storeAppID, bidder2.get_address()))

The smart contract now holds the following: {0: 1705001, 2989: 0}
storeAppID 3283
{b'SA': 1496000}
{}
{}
{b'BA': 1496000}


In [18]:
#bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]
bidder1NftBalance = get_balances(client, bidder1.get_address())[asset_id]
#bidder2NftBalance = get_balances(client, bidder2.get_address())[asset_id]

print("Carla's NFT balance:", bidder1NftBalance, " for token ID: ", asset_id)

assert bidder1NftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 0  for token ID:  2989


AssertionError: 

In [19]:
storeAppState = get_app_global_state(client, storeAppID)
print(storeAppState)
print(encoding.encode_address(storeAppState[b"C"]))

{b'AA_ADDR': 3285, b'BA_ADDR': 101, b'TA_ADDR': 100, b'TBA': 1496000, b'TSA': 1496000}


KeyError: b'C'